In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
from official.nlp import optimization
from jcarbon.tensorflow.callbacks import JCarbonEpochCallback, JCarbonBatchCallback, JCarbonChunkingCallback

import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import keras
import time

tf.get_logger().setLevel('ERROR')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

2024-06-12 00:03:24.058377: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 00:03:24.111226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 00:03:24.111267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 00:03:24.112822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 00:03:24.120931: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
#add bert model
bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'
#bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [3]:
def make_bert_preprocess_model(sentence_features, seq_length=128):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load(tfhub_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return tf.keras.Model(input_segments, model_inputs)

In [4]:
AUTOTUNE = tf.data.AUTOTUNE


def load_dataset_from_tfds(in_memory_ds, info, split, batch_size,
                           bert_preprocess_model):
    is_training = split.startswith('train')
    dataset = tf.data.Dataset.from_tensor_slices(in_memory_ds[split])
    num_examples = info.splits[split].num_examples

  # dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
    # iterator = iter(dataset)
    # while True:
    #     try:
    #         _ = next(iterator)
    #     except tf.errors.OutOfRangeError:
    #         break
    for _ in dataset:
        pass
    dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
    if is_training:
        dataset = dataset.shuffle(num_examples)
        dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex['label']))
  
    return dataset, num_examples

In [5]:
def build_classifier_model(num_classes):

  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [6]:
def get_configuration(glue_task):

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  if glue_task == 'glue/cola':
    metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)
  else:
    metrics = tf.keras.metrics.SparseCategoricalAccuracy(
        'accuracy', dtype=tf.float32)

  return metrics, loss

In [7]:
test_preprocess_model = make_bert_preprocess_model(['my_input1', 'my_input2'])
test_text = [np.array(['some random test sentence']),
             np.array(['another sentence'])]
text_preprocessed = test_preprocess_model(test_text)

print('Keys           : ', list(text_preprocessed.keys()))
print('Shape Word Ids : ', text_preprocessed['input_word_ids'].shape)
print('Word Ids       : ', text_preprocessed['input_word_ids'][0, :16])
print('Shape Mask     : ', text_preprocessed['input_mask'].shape)
print('Input Mask     : ', text_preprocessed['input_mask'][0, :16])
print('Shape Type Ids : ', text_preprocessed['input_type_ids'].shape)
print('Type Ids       : ', text_preprocessed['input_type_ids'][0, :16])

2024-06-12 00:03:27.885284: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15659 MB memory:  -> device: 0, name: Quadro P5000, pci bus id: 0000:18:00.0, compute capability: 6.1


Keys           :  ['input_mask', 'input_word_ids', 'input_type_ids']
Shape Word Ids :  (1, 128)
Word Ids       :  tf.Tensor(
[ 101 2070 6721 3231 6251  102 2178 6251  102    0    0    0    0    0
    0    0], shape=(16,), dtype=int32)
Shape Mask     :  (1, 128)
Input Mask     :  tf.Tensor([1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0], shape=(16,), dtype=int32)
Shape Type Ids :  (1, 128)
Type Ids       :  tf.Tensor([0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0], shape=(16,), dtype=int32)


In [8]:
#index = ['mrpc', 'cola', 'sst2', 'qqp', 'stsb', 'mnli', 'qnli', 'rte', 'wnli']
#index = ['wnli', 'rte', 'mrpc', 'cola', 'sst2','qnli','qqp','mnli','sst2']
index = ['cola','mrpc','wnli']
total_cpu = []
total_gpu = []
tfds_prefix = 'glue/'

In [9]:
def bar_chart_plot(jcarb):
    signal = 'jcarbon.emissions.Emissions'
    ax = None
    x = 0
    df1 = pd.DataFrame({'cpu_emissions':[], 'gpu_emissions':[], 'epochs':[]});
    for report in jcarb.reports:
        x+=1
        df = report.reset_index()
        df = df[df.signal == signal]
        df = df[df['value'] >= 0]
        df['cpu'] = df.subcomponent
        groups = df.groupby('cpu')
        cpu_df = groups.get_group('socket=0')
        gpu_df = groups.get_group('device=0:name=Quadro P5000')
        new_row = pd.DataFrame({'cpu_emissions': [cpu_df['value'].sum()], 'gpu_emissions':[gpu_df['value'].sum()], 'epochs':[x]});
        df1 = pd.concat([new_row,df1])
    cpu_sum = df1['cpu_emissions'].sum()
    gpu_sum = df1['gpu_emissions'].sum()
    return cpu_sum, gpu_sum

In [10]:
#stacked line plot for cpu + gpu
def stacked_line_plot(jcarb, dataset):
    signal = 'jcarbon.emissions.Emissions'
    ax = None
    x = 0
    df1 = pd.DataFrame({'cpu_emissions':[], 'gpu_emissions':[], 'epochs':[]});
    for report in jcarb.reports:
        x+=1
        df = report.reset_index()
        df = df[df.signal == signal]
        df = df[df['value'] >= 0]
        df['cpu'] = df.subcomponent
        groups = df.groupby('cpu')
        cpu_df = groups.get_group('socket=0')
        gpu_df = groups.get_group('device=0:name=Quadro P5000')
        new_row = pd.DataFrame({'cpu_emissions': [cpu_df['value'].sum()], 'gpu_emissions':[gpu_df['value'].sum()], 'epochs':[x]});
        df1 = pd.concat([new_row,df1])
    params = bert_model_name.strip('small_bert/bert_en_uncased_')
    path_name = f'tf_plots/line_plots/{dataset}-plot.png'
    raw_path = f'tf_plots/line_plots/raw_data/{dataset}.csv'
    df1.to_csv(raw_path, index = False)
    ax = df1.plot(x = 'epochs', stacked = True, xlabel = 'epochs', ylabel = 'emissions (grams of co2)', title = f'{tfds_name} with {params}') 
    ax.locator_params(integer=True)
    ax.figure.savefig(path_name)
# ax1 = df.plot(x = 'epochs', y = 'total_emissions', xlabel = 'epoch', ylabel = 'emissions (grams of co2)', title = tfds_name)

In [11]:
#time series plot
def time_series_plot(jcarb, dataset):
    signal = 'jcarbon.emissions.Emissions'
    ax=None
    for report in jcarb.reports:
        df = report.reset_index()
        df = df[df.signal == signal]
        df = df[df['value'] >= 0]
        df['elapsed'] = (df.start - df.start.min()).dt.total_seconds()
        df['rolled_value'] = df['value'].rolling(1000).mean()
        ax = df.plot(x='elapsed', y='rolled_value', xlabel = 'elapsed time', ylabel = 'emissions (grams of co2)', legend=False, figsize=(16, 9), ax=ax)
    params = bert_model_name.strip('small_bert/bert_en_uncased_')
    path_name = f'tf_plots/time_series/{dataset}-plot-series.png'
    raw_path = f'tf_plots/time_series/raw_data/{dataset}.csv'
    df.to_csv(raw_path, index = False)
    ax.figure.savefig(path_name)

In [12]:
signal = ['jcarbon.emissions.Emissions', 'jcarbon.server.MonotonicTimestamp']

In [13]:
def normalize_timestamps(timestamps, bucket_size_ms):
    """ normalizes ns timestamps to ms-bucketed timestamps """
    # TODO: this is producing strange behavior due to int division:
    #   2938450289096200 // 10**6 = 2938450288
    # TODO: taken from vesta's source. need to determine how to merge
    return bucket_size_ms * (timestamps // 10**6 // bucket_size_ms)

In [14]:
def to_dataframe(report, signals=None):
    signals_df = []
    monotonic_time = {}
    for jcarbon_signal in report.signal:
        if jcarbon_signal.signal_name == 'jcarbon.server.MonotonicTimestamp':
            # TODO: for now, i'm always grabbing the monotonic time.
            monotonic_time = {}
            for signal in jcarbon_signal.signal:
                start = 1000000000 * signal.start.secs + signal.start.nanos
                for data in signal.data:
                    monotonic_time[start] = data.value
            monotonic_time = pd.Series(monotonic_time)
            monotonic_time.index.name = 'timestamp'
            monotonic_time.name = 'ts'
        elif signals is None or jcarbon_signal.signal_name in signals:
            df = []
            for signal in jcarbon_signal.signal:
                start = 1000000000 * signal.start.secs + signal.start.nanos
                end = 1000000000 * signal.end.secs + signal.end.nanos
                for data in signal.data:
                    df.append([
                        jcarbon_signal.signal_name,
                        start,
                        end,
                        signal.component.replace(',', ':'),
                        signal.unit,
                        data.component.replace(',', ':'),
                        data.value,
                    ])
            signals_df.append(pd.DataFrame(data=df, columns=[
                              'signal', 'start', 'end', 'component', 'unit', 'subcomponent', 'value']))

    signals_df = pd.concat(signals_df)
    diff = (signals_df.end - signals_df.start).min() // 1000
    signals_df['start_norm'] = normalize_timestamps(signals_df.start, diff)
    # monotonic_time.index = normalize_timestamps(monotonic_time.index, diff)
    signals_df['ts'] = 0
    signals_df['start'] = pd.to_datetime(signals_df.start, unit='ns')
    signals_df['end'] = pd.to_datetime(signals_df.end, unit='ns')

    return signals_df.set_index(['signal', 'start', 'end', 'ts', 'component', 'unit', 'subcomponent']).value.sort_index()

In [15]:
from tensorflow.keras.callbacks import Callback

from jcarbon.client import JCarbonClient
# from jcarbon.report import to_dataframe

DEFAULT_PERIOD_MS = 10
DEFAULT_SIGNALS = [
    'jcarbon.cpu.eflect.ProcessEnergy',
    'jcarbon.emissions.Emissions',
    'jcarbon.server.MonotonicTimestamp',
    'jcarbon.nvml.NvmlEstimatedEnergy',
    'jcarbon.nvml.NvmlTotalEnergy',
]

In [16]:
class JCarbonChunkingCallback(Callback):
    def __init__(self, addr='localhost:8980', period_ms=DEFAULT_PERIOD_MS, signals=DEFAULT_SIGNALS):
        self.pid = os.getpid()
        self.period_ms = period_ms
        self.client = JCarbonClient(addr)
        self.signals = signals
        self.reports = []
        print("hello")

    def on_epoch_begin(self, epoch, logs=None):
        self.time = time.time()
        self.chunks = []
        self.client.start(self.pid, self.period_ms)

    def on_train_batch_end(self, epoch, logs=None):
        curr = time.time()
        if(curr - self.time > 10):
            self.client.stop(self.pid)
            self.chunks.append(to_dataframe(
                self.client.read(self.pid, self.signals)))
            self.client.start(self.pid, self.period_ms)
            self.time = curr
            if logs is not None:
                for (signal, component, unit), df in self.chunks[-1].groupby(['signal', 'component', 'unit']):
                    # TODO: this should really not ignore negatives
                    logs[f'jcarbon-epoch-{signal}-{component}-{unit}'] = df[df > 0].sum()
                
    def on_epoch_end(self, epoch, logs=None):
        self.client.stop(self.pid)
        self.chunks.append(to_dataframe(
            self.client.read(self.pid, self.signals)))
        self.reports.append(pd.concat(self.chunks))
        if logs is not None:
            for (signal, component, unit), df in self.reports[-1].groupby(['signal', 'component', 'unit']):
                # TODO: this should really not ignore negatives
                logs[f'jcarbon-epoch-{signal}-{component}-{unit}'] = df[df > 0].sum()
    

In [ ]:
for dataset in index:
    text_classifier_model = build_classifier_model(2)
    bert_raw_result = text_classifier_model(text_preprocessed)
    print(tf.sigmoid(bert_raw_result))

    tfds_name = f'{tfds_prefix}{dataset}'
    tfds_info = tfds.builder(tfds_name).info

    sentence_features = list(tfds_info.features.keys())
    sentence_features.remove('idx')
    sentence_features.remove('label')
    
    available_splits = list(tfds_info.splits.keys())
    train_split = 'train'
    validation_split = 'validation'
    test_split = 'test'
    if tfds_name == 'glue/mnli':
      validation_split = 'validation_matched'
      test_split = 'test_matched'
        
    try:
        num_classes = tfds_info.features['label'].num_classes
        print(f'Number of classes: {num_classes}')
    except AttributeError:
        print("num classes doesn't exist in {tfds_name}")
    try:
        num_examples = tfds_info.splits.total_num_examples
        print(f'This dataset has {num_examples} examples')
    except AttributeError:
        print("num examples doesn't exist in {tfds_name}")
    
        
    print(f'Using {tfds_name} from TFDS')
    print(f'Features {sentence_features}')
    print(f'Splits {available_splits}\n')
    
    with tf.device('/job:localhost'):
      # batch_size=-1 is a way to load the dataset into memory
      in_memory_ds = tfds.load(tfds_name, batch_size=-1, shuffle_files=True)

        #loading datasets
    batch_size = 32
    
    print(f'Fine tuning {tfhub_handle_encoder} model')
    bert_preprocess_model = make_bert_preprocess_model(sentence_features)
    
    train_ds, train_ds_size = load_dataset_from_tfds(
           in_memory_ds, tfds_info, train_split, batch_size, bert_preprocess_model)
    
    
    val_ds, val_ds_size = load_dataset_from_tfds(
          in_memory_ds, tfds_info, validation_split, batch_size,
          bert_preprocess_model)
    
    validation_steps = val_ds_size // batch_size

    #create loss function
    metrics, loss = get_configuration(tfds_name)
    
    #create finetuning/optimizer
    epochs = 100
    steps_per_epoch = train_ds_size #batch size
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = 10
    
    init_lr = 2e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')
    #load and train BERT
    classifier_model = build_classifier_model(num_classes)
    classifier_model.compile(optimizer=optimizer,
                             loss=loss,
                             metrics=metrics)

    #do the actual training
    print(f'Training model with {tfhub_handle_encoder}')
    jcarb = JCarbonChunkingCallback(addr='localhost:8981', period_ms=1000, signals=signal)
    
    classifier_model.fit(x=train_ds,
                           validation_data=val_ds,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=val_ds_size,
                           callbacks = jcarb)
    cpu_emissions, gpu_emissions = bar_chart_plot(jcarb)
    stacked_line_plot(jcarb, dataset)
    time_series_plot(jcarb, dataset)
    
    total_cpu.append(cpu_emissions)
    total_gpu.append(gpu_emissions)

tf.Tensor([[0.29277614 0.25735623]], shape=(1, 2), dtype=float32)
Number of classes: 2
This dataset has 10657 examples
Using glue/cola from TFDS
Features ['sentence']
Splits ['train', 'validation', 'test']

Fine tuning https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1 model


/home/vincent/.local/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['idx', 'label'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
hello
Epoch 1/100
8550/8551 [============================>.] - ETA: 0s - loss: 0.5256 - MatthewsCorrelationCoefficient: 0.0000e+00 - jcarbon-epoch-jcarbon.nvml.NvmlEstimatedEnergy-nvml-JOULES: 757.4418 - jcarbon-epoch-jcarbon.nvml.NvmlTotalEnergy-nvml-JOULES: 786.3670

2024-06-12 00:07:31.725581: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16812416964289065126
2024-06-12 00:07:31.725669: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16438108324085350556
2024-06-12 00:07:31.725687: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10313258542156104508
2024-06-12 00:07:31.725702: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6265514883361153838
2024-06-12 00:07:31.725716: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6340696860084209936
2024-06-12 00:07:31.725730: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8716960273430886980
2024-06-12 00:07:31.725748: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

8551/8551 [==============================] - 231s 26ms/step - loss: 0.5256 - MatthewsCorrelationCoefficient: 0.0000e+00 - jcarbon-epoch-jcarbon.nvml.NvmlEstimatedEnergy-nvml-JOULES: 1479.9150 - jcarbon-epoch-jcarbon.nvml.NvmlTotalEnergy-nvml-JOULES: 1536.2424 - val_loss: 0.8142 - val_MatthewsCorrelationCoefficient: 0.0000e+00
Epoch 2/100
8551/8551 [==============================] - 217s 25ms/step - loss: 0.3247 - MatthewsCorrelationCoefficient: 0.0000e+00 - jcarbon-epoch-jcarbon.nvml.NvmlEstimatedEnergy-nvml-JOULES: 1395.6074 - jcarbon-epoch-jcarbon.nvml.NvmlTotalEnergy-nvml-JOULES: 1448.1011
Epoch 3/100
6342/8551 [=====================>........] - ETA: 55s - loss: 0.2305 - MatthewsCorrelationCoefficient: 0.0000e+00 - jcarbon-epoch-jcarbon.nvml.NvmlEstimatedEnergy-nvml-JOULES: 722.7337 - jcarbon-epoch-jcarbon.nvml.NvmlTotalEnergy-nvml-JOULES: 750.9195

In [ ]:
df = pd.DataFrame({'total_cpu_emissions': total_cpu,
                   'total_gpu_emissions': total_gpu,
                   'dataset':index}, index=index)
ax = df.plot.bar(stacked = True, ylabel = 'emissions (grams of co2)')
ax.figure.savefig('tf_plots/bar_chart.png')
raw_path = f'tf_plots/plot_data.csv'
df.to_csv(raw_path, index=False) 